# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fda6d6b2bb0>
├── 'a' --> tensor([[ 2.7540, -2.9680, -0.4019],
│                   [-0.1188,  0.6198, -0.6591]])
└── 'x' --> <FastTreeValue 0x7fda6d6b2df0>
    └── 'c' --> tensor([[-0.3648,  1.8585,  0.1400, -1.8218],
                        [-1.8204,  0.4281,  0.3082,  0.9682],
                        [ 0.2230,  0.0945,  2.0807,  1.0309]])

In [4]:
t.a

tensor([[ 2.7540, -2.9680, -0.4019],
        [-0.1188,  0.6198, -0.6591]])

In [5]:
%timeit t.a

63.2 ns ± 0.115 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fda6d6b2bb0>
├── 'a' --> tensor([[ 0.4885,  1.5179,  0.4968],
│                   [ 1.3098, -0.6284,  1.0075]])
└── 'x' --> <FastTreeValue 0x7fda6d6b2df0>
    └── 'c' --> tensor([[-0.3648,  1.8585,  0.1400, -1.8218],
                        [-1.8204,  0.4281,  0.3082,  0.9682],
                        [ 0.2230,  0.0945,  2.0807,  1.0309]])

In [7]:
%timeit t.a = new_value

71.8 ns ± 0.0544 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 2.7540, -2.9680, -0.4019],
               [-0.1188,  0.6198, -0.6591]]),
    x: Batch(
           c: tensor([[-0.3648,  1.8585,  0.1400, -1.8218],
                      [-1.8204,  0.4281,  0.3082,  0.9682],
                      [ 0.2230,  0.0945,  2.0807,  1.0309]]),
       ),
)

In [10]:
b.a

tensor([[ 2.7540, -2.9680, -0.4019],
        [-0.1188,  0.6198, -0.6591]])

In [11]:
%timeit b.a

58.9 ns ± 0.03 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.1808,  1.9419, -0.2587],
               [ 0.4704,  0.1879,  0.7427]]),
    x: Batch(
           c: tensor([[-0.3648,  1.8585,  0.1400, -1.8218],
                      [-1.8204,  0.4281,  0.3082,  0.9682],
                      [ 0.2230,  0.0945,  2.0807,  1.0309]]),
       ),
)

In [13]:
%timeit b.a = new_value

500 ns ± 1.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

881 ns ± 3.04 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 11.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 329 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd9c4f27970>
├── 'a' --> tensor([[[ 2.7540, -2.9680, -0.4019],
│                    [-0.1188,  0.6198, -0.6591]],
│           
│                   [[ 2.7540, -2.9680, -0.4019],
│                    [-0.1188,  0.6198, -0.6591]],
│           
│                   [[ 2.7540, -2.9680, -0.4019],
│                    [-0.1188,  0.6198, -0.6591]],
│           
│                   [[ 2.7540, -2.9680, -0.4019],
│                    [-0.1188,  0.6198, -0.6591]],
│           
│                   [[ 2.7540, -2.9680, -0.4019],
│                    [-0.1188,  0.6198, -0.6591]],
│           
│                   [[ 2.7540, -2.9680, -0.4019],
│                    [-0.1188,  0.6198, -0.6591]],
│           
│                   [[ 2.7540, -2.9680, -0.4019],
│                    [-0.1188,  0.6198, -0.6591]],
│           
│                   [[ 2.7540, -2.9680, -0.4019],
│                    [-0.1188,  0.6198, -0.6591]]])
└── 'x' --> <FastTreeValue 0x7fda686282b0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 115 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fda6d709820>
├── 'a' --> tensor([[ 2.7540, -2.9680, -0.4019],
│                   [-0.1188,  0.6198, -0.6591],
│                   [ 2.7540, -2.9680, -0.4019],
│                   [-0.1188,  0.6198, -0.6591],
│                   [ 2.7540, -2.9680, -0.4019],
│                   [-0.1188,  0.6198, -0.6591],
│                   [ 2.7540, -2.9680, -0.4019],
│                   [-0.1188,  0.6198, -0.6591],
│                   [ 2.7540, -2.9680, -0.4019],
│                   [-0.1188,  0.6198, -0.6591],
│                   [ 2.7540, -2.9680, -0.4019],
│                   [-0.1188,  0.6198, -0.6591],
│                   [ 2.7540, -2.9680, -0.4019],
│                   [-0.1188,  0.6198, -0.6591],
│                   [ 2.7540, -2.9680, -0.4019],
│                   [-0.1188,  0.6198, -0.6591]])
└── 'x' --> <FastTreeValue 0x7fda6d709880>
    └── 'c' --> tensor([[-0.3648,  1.8585,  0.1400, -1.8218],
                        [-1.8204,  0.4281,  0.3082,  0.9682],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 60.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.7 µs ± 37 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 2.7540, -2.9680, -0.4019],
                [-0.1188,  0.6198, -0.6591]],
       
               [[ 2.7540, -2.9680, -0.4019],
                [-0.1188,  0.6198, -0.6591]],
       
               [[ 2.7540, -2.9680, -0.4019],
                [-0.1188,  0.6198, -0.6591]],
       
               [[ 2.7540, -2.9680, -0.4019],
                [-0.1188,  0.6198, -0.6591]],
       
               [[ 2.7540, -2.9680, -0.4019],
                [-0.1188,  0.6198, -0.6591]],
       
               [[ 2.7540, -2.9680, -0.4019],
                [-0.1188,  0.6198, -0.6591]],
       
               [[ 2.7540, -2.9680, -0.4019],
                [-0.1188,  0.6198, -0.6591]],
       
               [[ 2.7540, -2.9680, -0.4019],
                [-0.1188,  0.6198, -0.6591]]]),
    x: Batch(
           c: tensor([[[-0.3648,  1.8585,  0.1400, -1.8218],
                       [-1.8204,  0.4281,  0.3082,  0.9682],
                       [ 0.2230,  0.0945,  2.0807,  1.0309]],
         

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 238 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 2.7540, -2.9680, -0.4019],
               [-0.1188,  0.6198, -0.6591],
               [ 2.7540, -2.9680, -0.4019],
               [-0.1188,  0.6198, -0.6591],
               [ 2.7540, -2.9680, -0.4019],
               [-0.1188,  0.6198, -0.6591],
               [ 2.7540, -2.9680, -0.4019],
               [-0.1188,  0.6198, -0.6591],
               [ 2.7540, -2.9680, -0.4019],
               [-0.1188,  0.6198, -0.6591],
               [ 2.7540, -2.9680, -0.4019],
               [-0.1188,  0.6198, -0.6591],
               [ 2.7540, -2.9680, -0.4019],
               [-0.1188,  0.6198, -0.6591],
               [ 2.7540, -2.9680, -0.4019],
               [-0.1188,  0.6198, -0.6591]]),
    x: Batch(
           c: tensor([[-0.3648,  1.8585,  0.1400, -1.8218],
                      [-1.8204,  0.4281,  0.3082,  0.9682],
                      [ 0.2230,  0.0945,  2.0807,  1.0309],
                      [-0.3648,  1.8585,  0.1400, -1.8218],
                      [-1.8204,  

In [28]:
%timeit Batch.cat(batches)

145 µs ± 290 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

327 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
